In [ ]:
!pip install deep-kan

In [ ]:
from deepkan import RBFKAN

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Load MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
valset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)

# Define model
model = RBFKAN(layers_hidden=[28 * 28, 64, 10])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

# Define loss
criterion = nn.CrossEntropyLoss()

# Define ReduceLROnPlateau scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

for epoch in range(20):
    # Train
    model.train()
    total_loss = 0
    total_accuracy = 0
    with tqdm(trainloader) as pbar:
        for images, labels in pbar:
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            accuracy = (output.argmax(dim=1) == labels).float().mean()
            total_loss += loss.item()
            total_accuracy += accuracy.item()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item())
    total_loss /= len(trainloader)
    total_accuracy /= len(trainloader)

    # Validation
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)
            output = model(images)
            val_loss += criterion(output, labels).item()
            val_accuracy += (output.argmax(dim=1) == labels).float().mean().item()
    val_loss /= len(valloader)
    val_accuracy /= len(valloader)

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1}, Train Loss: {total_loss}, Train Accuracy: {total_accuracy}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")

100%|██████████| 938/938 [00:16<00:00, 57.15it/s, accuracy=0.906, loss=0.149] 


Epoch 1, Train Loss: 0.4723579772213883, Train Accuracy: 0.8596748400852878, Val Loss: 0.28278024633456567, Val Accuracy: 0.9156050955414012


100%|██████████| 938/938 [00:15<00:00, 58.91it/s, accuracy=0.969, loss=0.096] 


Epoch 2, Train Loss: 0.27193666877808853, Train Accuracy: 0.9211587153518124, Val Loss: 0.23299388688318667, Val Accuracy: 0.9289410828025477


100%|██████████| 938/938 [00:15<00:00, 59.63it/s, accuracy=0.906, loss=0.405] 


...

Epoch 18, Train Loss: 0.069909372185764, Train Accuracy: 0.9778784648187633, Val Loss: 0.1489242538342632, Val Accuracy: 0.9583996815286624


100%|██████████| 938/938 [00:17<00:00, 53.90it/s, accuracy=1, loss=0.0293]    


Epoch 19, Train Loss: 0.06813243783931973, Train Accuracy: 0.9772454690831557, Val Loss: 0.14044109393900356, Val Accuracy: 0.9610867834394905


100%|██████████| 938/938 [00:16<00:00, 57.02it/s, accuracy=0.938, loss=0.121] 


Epoch 20, Train Loss: 0.06659237002563685, Train Accuracy: 0.9781783049040512, Val Loss: 0.15983872259499884, Val Accuracy: 0.9579020700636943


# It is faster than normal KANs with a speed of 60 it/s